In [5]:
# Bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import unicodedata
import re
import os
import numpy as np
from collections import Counter
from scipy.spatial.distance import euclidean
import string
import spacy
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm, tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from nltk.corpus import stopwords
nltk.download('stopwords')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import scipy.stats as stats
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, get_linear_schedule_with_warmup
from torch.optim import AdamW
from tqdm import tqdm

[nltk_data] Downloading package stopwords to /home/ester/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df_clean = pd.read_csv("df_ex1_clean.csv")
display(df_clean)

,Label,Message,Word_count,Message_clean,Word_count_clean
0,ham,"Go until jurong point, crazy.. Available only ...",20,crazy available bugis great world buffet cine ...,10
1,ham,Ok lar... Joking wif u oni...,6,lar joke wif oni,4
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,28,free entry wkly comp win cup final tkts text r...,15
3,ham,U dun say so early hor... U c already then say...,11,early hor,2
4,ham,"Nah I don't think he goes to usf, he lives aro...",13,nah think go usf life,5
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,30,time try contact pound prize claim easy minute...,9
5568,ham,Will ü b going to esplanade fr home?,8,go esplanade home,3
5569,ham,"Pity, * was in mood for that. So...any other s...",10,pity mood soany suggestion,4
5570,ham,The guy did some bitching but I acted like i'd...,26,guy bitching act like interested buy week give...,9


In [6]:
df_clean['Message_clean'] = df_clean['Message_clean'].fillna('').astype(str)


### Passo 8: Modelos Transformers

BERT Zero-shot - Extração de Features

In [ ]:
# Preparação dos dados
texts = df_clean['Message_clean'].tolist()
labels_texto = df_clean['Label'].tolist()

encoder = LabelEncoder()
labels = encoder.fit_transform(labels_texto)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Tenta usar a gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval() # Coloca o modelo em modo de avaliação, não de treino

In [ ]:
# Essa classe cuida da tokenização do texto, tratamento do comprimento da sequência e fornece um pacote
# organizado com IDs de entrada, máscaras de atêncção e rótulos para o modelo aprender 
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
            self.texts = texts
            self.labels = labels
            self.tokenizer = tokenizer
            self.max_length = max_length
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
        return {'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'label': torch.tensor(label)}

# Cria uma instância do dataset
meu_dataset = TextClassificationDataset(texts, labels, tokenizer, max_length=128)

# Entrega o dataset para o DataLoader
meu_dataloader = DataLoader(meu_dataset, batch_size=16, shuffle=True)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [8]:
print("Iniciando a extração de features do BERT")

# Listas para guardar os resultados de cada lote
lista_de_vetores = []
lista_de_rotulos = []

# Loop de extração (sem treinamento, pois é o zero shot)
# torch.no_grad() desativa o cálculo de gradientes, o que economiza
# memória e acelera o processo
with torch.no_grad():
    for batch in tqdm(meu_dataloader, desc="Extraindo vetores BERT"):
        
        # Mover os dados do lote para a GPU
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label']

        # Passar os dados pelo modelo BERT
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        
        # A saída que quero é a 'last_hidden_state'.
        # Ela contém os vetores de saída para todos os tokens de todas as frases do lote.
        # O formato é: [tamanho_do_lote, tamanho_da_sequencia, tamanho_do_vetor]
        # Queremos o vetor do token [CLS], que é sempre o primeiro (índice 0).
        vetores_cls = outputs.last_hidden_state[:, 0, :]
        
        # Adicionar os resultados do lote às nossas listas.
        # É importante mover os tensores de volta para a CPU com .cpu()
        lista_de_vetores.append(vetores_cls.cpu())
        lista_de_rotulos.append(labels)

# Juntar os resultados de todos os lotes em tensores únicos
X_bert = torch.cat(lista_de_vetores, dim=0)
y_bert = torch.cat(lista_de_rotulos, dim=0)

# Converter para o formato NumPy, que é o que o scikit-learn usa
X_bert_numpy = X_bert.numpy() # Matriz onde cada linha é o vetor de 768 dimensões que representa uma das mensagens
y_bert_numpy = y_bert.numpy() # Um vetor com os rótulos numéricos correspondentes

print("\nExtração concluída!")
print("Formato da matriz de features (X):", X_bert_numpy.shape)
print("Formato do vetor de rótulos (y):", y_bert_numpy.shape)

Iniciando a extração de features do BERT


Extraindo vetores BERT: 100%|██████████| 349/349 [01:27<00:00,  3.98it/s]


Extração concluída!
Formato da matriz de features (X): (5572, 768)
Formato do vetor de rótulos (y): (5572,)


## Utilizando a representação gerada pelo BERT nos classificadores novamente

In [9]:
# Árvore de decisão

def arvore_de_decisao(X_train, y_train, X_test, y_test):
    parametros = {
        'criterion': ['gini', 'entropy'], # Função usada para medir a qualidade de uma divisão
        'max_depth': [None, 10, 20], # Profundidade máxima 
        'min_samples_split': [2, 5, 10], # Mínimo de amostras exigido para dividir um nó
        'min_samples_leaf': [1, 2, 4] # Mínimo de amostras exigido em um nó folha
    }

    # Treinando a árvore de decisão
    clf = tree.DecisionTreeClassifier()

    grid_search = GridSearchCV(estimator=clf, param_grid=parametros, cv=5, scoring='accuracy')
    #cv =5 significa que os dados sao divididos em 5 partes. O modelo é treinado 5 vezes, utilizando 4 partes
    # para treino.

    grid_search.fit(X_train, y_train) # Treina o modelo utilizando os melhores parametros encontrados

    print("Melhores parâmetros: ", grid_search.best_params_)

    melhor_clf = grid_search.best_estimator_ # treina novamente com os melhores parametros
    y_pred = melhor_clf.predict(X_test) # Fazendo a previsão no conjunto de teste

    acuracia = accuracy_score(y_test, y_pred)
    f1_macro = f1_score(y_test, y_pred, average='macro')

    print(f"\nAcurácia com os melhores parâmetros: {acuracia}")
    print(f"F1 Score (macro) com os melhores parâmetros: {f1_macro}")

    #plt.figure(figsize=(20,10))
    #tree.plot_tree(melhor_clf, max_depth=2, feature_names=vectorizer.get_feature_names_out(), class_names=codificador.classes_, filled=True)
    #plt.show()
    # max_depth profundidade da árvore
    # filled=True colore os nós de acordo com a classe predominante
    # feature_names mostra quais palavras sao mais importantes para as decisoes

    return grid_search.best_params_


In [10]:
# KNN - K-Nearest Neighbor

def knn(X_train, y_train, X_test, y_test):
    parametros = {'n_neighbors': [1, 3, 5, 7, 9],
                'weights': ['uniform', 'distance'],
                'metric': ['euclidean', 'manhattan']}

    # Aplicando KNN
    knn = KNeighborsClassifier()

    grid_search = GridSearchCV(estimator=knn, param_grid=parametros, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train) # Treina o modelo com diferentes combinações
    print("Melhores parâmetros: ", grid_search.best_params_)

    melhor_knn = grid_search.best_estimator_
    melhor_y_pred = melhor_knn.predict(X_test) # Realiza predições com o modelo ajustado

    acuracia = accuracy_score(y_test, melhor_y_pred)
    f1_macro = f1_score(y_test, melhor_y_pred, average='macro')

    print(f"Acurácia com os melhores parâmetros: {acuracia}")
    print(f"F1 Score (macro) com os melhores parâmetros: {f1_macro}")

    return grid_search.best_params_



In [12]:
# SVM 

def svm_modelo(X_train, y_train, X_test, y_test):
    parametros = {
        'C': [1, 10],  # Parâmetro C
        'gamma': [0.1, 'auto', 'scale'],  # Parâmetro gamma
        'kernel': ['rbf', 'linear']  # Tipos de kernel
    }
    # Parâmetro C pequeno tolera mais erros (evita overfitting) - está com valor padrão
    # Parâmetro gamma pequeno define que cada ponto de treino tem influencia ampla resultando em 
    # uma fronteira maior - está com valor padrão

    svc = svm.SVC()

    grid_search = GridSearchCV(estimator=svc, param_grid=parametros, cv=5, scoring='accuracy')

    grid_search.fit(X_train, y_train)

    print(f"Melhores parâmetros encontrados: {grid_search.best_params_}")

    # Fazendo a previsão com os melhores parametros
    melhor_svm = grid_search.best_estimator_
    y_pred = melhor_svm.predict(X_test)

    print(f"Acurácia: {accuracy_score(y_test, y_pred)}")
    f1_macro = f1_score(y_test, y_pred, average='macro')
    print(f"F1-Score Macro: {f1_macro}")

    return grid_search.best_params_




In [13]:
def calcular_intervalo_confianca(valores):
    media = np.mean(valores)
    desvio_padrao = np.std(valores)
    n = len(valores)

    t_critico = stats.t.ppf(0.975, df=n-1) # t Student (95% de confiança)

    erro_padrao = desvio_padrao/np.sqrt(n)

    intervalo_confianca = t_critico * erro_padrao

    return media, desvio_padrao, intervalo_confianca

def comparar_modelos_teste_t(resultados):
    # Converte as chaves do dicionário 'resultados' em uma lista de modelos
    modelos = list(resultados.keys())
    acuracias = [resultados[modelo] for modelo in modelos]

    comparacoes = []
    valores_p = []

    # Comparar todos os pares de modelos com o teste t
    for i in range(len(modelos)):
        for j in range(i+1, len(modelos)):
            # Realiza o Teste T entre os dois modelos e extrai o valor p
            _, valor_p = stats.ttest_ind(acuracias[i], acuracias[j])
            comparacoes.append(((modelos[i], modelos[j]), valor_p))
            print(f"Teste T entre {modelos[i]} e {modelos[j]}: valor_p = {valor_p:.4f}")

    # Correção de Bonferroni: ajusta o valor p para múltiplas comparações
    bonferroni_valores_p = [p * len(comparacoes) for p in comparacoes]

    # Loop para exibir os valores p corrigidos pelo método de Bonferroni
    for idx, ((modelo1, modelo2), valor_p) in enumerate(comparacoes):
        p_corrigido = valor_p * len(comparacoes)
        print(f"P-value corrigido Bonferroni ({modelo1} vs {modelo2}): {p_corrigido:.4f}")
        if p_corrigido < 0.05:
            print("Os métodos são estatisticamente diferentes.")
        else:
            print("Os métodos são estatisticamente equivalentes.")

In [14]:
# Holdout

def holdout(modelo_cls, X, y, rep, test_size=0.2, best_params=None):
    print("\n --- Holdout --- ")
    acuracias = []
    f1_scores = []

    for i in range(rep):
        print(f"\nRepetição {i}:")

        # Divide os dados diretamente
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=i)

        # Instancia o modelo com os melhores parâmetros (se houver)
        modelo = modelo_cls(**best_params) if best_params else modelo_cls()

        # Treinamento e predição
        modelo.fit(X_train, y_train)
        y_pred = modelo.predict(X_test)

        # Métricas
        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='macro')
        acuracias.append(acc)
        f1_scores.append(f1)

        print(f"  Acurácia: {acc}")
        print(f"  F1 Score (macro): {f1}")

    print("\nMÉDIA FINAL HOLDOUT:")
    acuracia_media, acuracia_desvio, acuracia_ic = calcular_intervalo_confianca(acuracias)
    f1_media, f1_desvio, f1_ic = calcular_intervalo_confianca(f1_scores)

    print(f"  Acurácia média após Holdout: {acuracia_media:.4f} (± {acuracia_ic:.4f})")
    print(f"  Desvio padrão da Acurácia: {acuracia_desvio:.4f}")
    print(f"  F1 Score (macro) médio: {f1_media:.4f} (± {f1_ic:.4f})")
    print(f"  Desvio padrão do F1 Score (macro): {f1_desvio:.4f}")

    return acuracias, f1_scores


In [15]:
# Stratified K Fold

def stratified_k_fold(modelo, X, y, best_params, k=10):
    print("\n --- Stratified K Fold ---")
    skf = StratifiedKFold(n_splits=k)
    acuracias = []
    f1_scores = []

    for i, (train_idx, test_idx) in enumerate(skf.split(X, y)):
        print(f"Fold {i}:")
        
        # Dividindo os dados em treino e teste
        X_fold_train, X_fold_test = X[train_idx], X[test_idx]
        y_fold_train, y_fold_test = y[train_idx], y[test_idx]

        # Inicializando o modelo com os melhores parâmetros
        clf_fold = modelo(**best_params) if (best_params) else modelo()
        
        # Treinando o modelo
        clf_fold.fit(X_fold_train, y_fold_train)

        # Predição
        y_pred_fold = clf_fold.predict(X_fold_test)

        # Calculando as métricas
        acc = accuracy_score(y_fold_test, y_pred_fold)
        f1 = f1_score(y_fold_test, y_pred_fold, average='macro')

        # Armazenando os resultados
        acuracias.append(acc)
        f1_scores.append(f1)

        # Exibindo os resultados por fold
        print(f"  Acurácia do Fold {i}: {acc}")
        print(f"  F1 Score (macro) do Fold {i}: {f1}")

    acuracia_media, acuracia_desvio, acuracia_ic = calcular_intervalo_confianca(acuracias)
    f1_media, f1_desvio, f1_ic = calcular_intervalo_confianca(f1_scores)

    # Exibindo as métricas médias
    print(f"\nAcurácia média após K-Fold: {acuracia_media:.4f} (± {acuracia_ic:.4f})")
    print(f"F1 Score (macro) médio após K-Fold: {f1_media:.4f} (± {f1_ic:.4f})")
    print(f"  Desvio padrão da Acurácia: {acuracia_desvio:.4f}")
    print(f"  Desvio padrão do F1 Score (macro): {f1_desvio:.4f}")

    return acuracias, f1_scores


In [16]:
# Divisão dos dados do BERT em treino e teste
X_train_bert, X_test_bert, y_train_bert, y_test_bert = train_test_split(X_bert_numpy, y_bert_numpy, test_size=0.2, random_state=0)

In [17]:
print("\n--- ÁRVORE DE DECISÃO ---")
melhores_parametros_arvore = arvore_de_decisao(X_train_bert, y_train_bert, X_test_bert, y_test_bert)
resultados_arvore_k_fold = stratified_k_fold(DecisionTreeClassifier, X_bert_numpy, y_bert_numpy, melhores_parametros_arvore, 10)
resultados_arvore_holdout = holdout(DecisionTreeClassifier, X_bert_numpy, y_bert_numpy, 3, 0.2, melhores_parametros_arvore)



--- ÁRVORE DE DECISÃO ---
Melhores parâmetros:  {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 5}

Acurácia com os melhores parâmetros: 0.9372197309417041
F1 Score (macro) com os melhores parâmetros: 0.8552764303620963

 --- Stratified K Fold ---
Fold 0:
  Acurácia do Fold 0: 0.9354838709677419
  F1 Score (macro) do Fold 0: 0.8564814814814814
Fold 1:
  Acurácia do Fold 1: 0.9229390681003584
  F1 Score (macro) do Fold 1: 0.8295820164068326
Fold 2:
  Acurácia do Fold 2: 0.9515260323159784
  F1 Score (macro) do Fold 2: 0.8942025621003313
Fold 3:
  Acurácia do Fold 3: 0.9371633752244165
  F1 Score (macro) do Fold 3: 0.8612504893064303
Fold 4:
  Acurácia do Fold 4: 0.940754039497307
  F1 Score (macro) do Fold 4: 0.8660208611226521
Fold 5:
  Acurácia do Fold 5: 0.9389587073608617
  F1 Score (macro) do Fold 5: 0.8644186712485682
Fold 6:
  Acurácia do Fold 6: 0.9443447037701975
  F1 Score (macro) do Fold 6: 0.8825637781994519
Fold 7:
  Acurácia do Fold 7: 0

In [18]:
print("\n--- KNN ---")
melhores_parametros_knn = knn(X_train_bert, y_train_bert, X_test_bert, y_test_bert)
resultados_knn_k_fold = stratified_k_fold(KNeighborsClassifier, X_bert_numpy, y_bert_numpy, melhores_parametros_knn or {}, 10)
resultados_knn_holdout = holdout(KNeighborsClassifier, X_bert_numpy, y_bert_numpy, 3, 0.2, melhores_parametros_knn)



--- KNN ---
Melhores parâmetros:  {'metric': 'euclidean', 'n_neighbors': 9, 'weights': 'distance'}
Acurácia com os melhores parâmetros: 0.9560538116591928
F1 Score (macro) com os melhores parâmetros: 0.9053724741023506

 --- Stratified K Fold ---
Fold 0:
  Acurácia do Fold 0: 0.9695340501792115
  F1 Score (macro) do Fold 0: 0.9376589578281185
Fold 1:
  Acurácia do Fold 1: 0.9516129032258065
  F1 Score (macro) do Fold 1: 0.9030121218753822
Fold 2:
  Acurácia do Fold 2: 0.9712746858168761
  F1 Score (macro) do Fold 2: 0.9390524127366233
Fold 3:
  Acurácia do Fold 3: 0.9605026929982047
  F1 Score (macro) do Fold 3: 0.916197067512857
Fold 4:
  Acurácia do Fold 4: 0.9640933572710951
  F1 Score (macro) do Fold 4: 0.9262486097134686
Fold 5:
  Acurácia do Fold 5: 0.9533213644524237
  F1 Score (macro) do Fold 5: 0.8975291520434734
Fold 6:
  Acurácia do Fold 6: 0.9748653500897666
  F1 Score (macro) do Fold 6: 0.9472537878787879
Fold 7:
  Acurácia do Fold 7: 0.9461400359066428
  F1 Score (macro)

In [19]:
print("\n--- SVM ---")
melhores_parametros_svm = svm_modelo(X_train_bert, y_train_bert, X_test_bert, y_test_bert)
resultados_svm_k_fold = stratified_k_fold(SVC, X_bert_numpy, y_bert_numpy, melhores_parametros_svm, 10)
resultados_svm_holdout = holdout(SVC, X_bert_numpy, y_bert_numpy, 3, 0.2, melhores_parametros_svm)



--- SVM ---
Melhores parâmetros encontrados: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
Acurácia: 0.9713004484304932
F1-Score Macro: 0.9317029845489487

 --- Stratified K Fold ---
Fold 0:
  Acurácia do Fold 0: 0.9731182795698925
  F1 Score (macro) do Fold 0: 0.9412396537562393
Fold 1:
  Acurácia do Fold 1: 0.9802867383512545
  F1 Score (macro) do Fold 1: 0.9558858407779272
Fold 2:
  Acurácia do Fold 2: 0.9820466786355476
  F1 Score (macro) do Fold 2: 0.9605881354013359
Fold 3:
  Acurácia do Fold 3: 0.9640933572710951
  F1 Score (macro) do Fold 3: 0.9151690526957051
Fold 4:
  Acurácia do Fold 4: 0.9784560143626571
  F1 Score (macro) do Fold 4: 0.952147766323024
Fold 5:
  Acurácia do Fold 5: 0.9712746858168761
  F1 Score (macro) do Fold 5: 0.933832264195771
Fold 6:
  Acurácia do Fold 6: 0.9820466786355476
  F1 Score (macro) do Fold 6: 0.9605881354013359
Fold 7:
  Acurácia do Fold 7: 0.9748653500897666
  F1 Score (macro) do Fold 7: 0.9448233895618703
Fold 8:
  Acurácia do Fold 8: 0.962

In [20]:
print("Testes Houldout: ")

resultados_comparacao_houldout = {
    'Árvore de Decisão': resultados_arvore_holdout[0],
    'KNN': resultados_knn_holdout[0],
    'SVM': resultados_svm_holdout[0]
}

comparar_modelos_teste_t(resultados_comparacao_houldout)

print("Testes K fold: ")

resultados_comparacao_k_fold = {
    'Árvore de Decisão': resultados_arvore_k_fold[0],
    'KNN': resultados_knn_k_fold[0],
    'SVM': resultados_svm_k_fold[0]
}

comparar_modelos_teste_t(resultados_comparacao_k_fold)

Testes Houldout: 
Teste T entre Árvore de Decisão e KNN: valor_p = 0.0004
Teste T entre Árvore de Decisão e SVM: valor_p = 0.0001
Teste T entre KNN e SVM: valor_p = 0.0079
P-value corrigido Bonferroni (Árvore de Decisão vs KNN): 0.0012
Os métodos são estatisticamente diferentes.
P-value corrigido Bonferroni (Árvore de Decisão vs SVM): 0.0003
Os métodos são estatisticamente diferentes.
P-value corrigido Bonferroni (KNN vs SVM): 0.0238
Os métodos são estatisticamente diferentes.
Testes K fold: 
Teste T entre Árvore de Decisão e KNN: valor_p = 0.0001
Teste T entre Árvore de Decisão e SVM: valor_p = 0.0000
Teste T entre KNN e SVM: valor_p = 0.0014
P-value corrigido Bonferroni (Árvore de Decisão vs KNN): 0.0002
Os métodos são estatisticamente diferentes.
P-value corrigido Bonferroni (Árvore de Decisão vs SVM): 0.0000
Os métodos são estatisticamente diferentes.
P-value corrigido Bonferroni (KNN vs SVM): 0.0043
Os métodos são estatisticamente diferentes.


## Classificação Zero Shot com BERT

In [6]:
from transformers import pipeline
import pandas as pd

texts = df_clean['Message_clean'].astype(str).fillna("").tolist()
labels_texto = df_clean['Label'].tolist()

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

candidate_labels = ["ham", "spam"]
predictions = []

for text in texts:
    result = classifier(text, candidate_labels)
    predicted_label = result['labels'][0]  
    predictions.append(predicted_label)


acc = accuracy_score(labels_texto, predictions)
print("Acurácia zero-shot:", acc)
print("\nRelatório de classificação:\n", classification_report(labels_texto, predictions))

Device set to use cuda:0


Acurácia zero-shot: 0.6782124910265613

Relatório de classificação:
               precision    recall  f1-score   support

         ham       0.86      0.75      0.80      4825
        spam       0.10      0.19      0.13       747

    accuracy                           0.68      5572
   macro avg       0.48      0.47      0.47      5572
weighted avg       0.76      0.68      0.71      5572



### Passo 9: Fine Tuning

Rodar em alguma máquina do laboratório. 

In [ ]:
# Carregando o modelo
model = BertModel.from_pretrained('bert-base-uncased')

num_labels = 2
# Definindo um cabeçalho classificador
classifier = nn.Linear(768, num_labels)

# Concatenando o codificador e o classificador
model = nn.Sequential(model, classifier)

# Definindo um otimizador e uma função de perda para o modelo.
# Usando perda de entropia cruzada para os classificadores
criterion = nn.CrossEntropyLoss()  
optimizer = AdamW(model.parameters(), lr=2e-5)

# Criando os objetos DataLoader a partir do conjunto de dados
train_loader = DataLoader(train_set, batch_size=16, shuffle=True)
test_loader = DataLoader(test_set, batch_size=16)

NameError: name 'BertModel' is not defined

In [ ]:
# Função de treinamento
def train(model, optimizer, train_loader, criterion):
    model.train()
    total_loss = 0

    for batch in train_loader:
        optimizer.zero_grad()
        input_ids, attention_mask, labels = batch 
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Training loss: {total_loss/len(train_loader)}')

In [ ]:
# Função de avaliação
def evaluate(model, test_loader, criterion):
    model.eval()
    total_loss = 0
    total_acc = 0

    with torch.no_grad():
        for batch in test_loader:
            input_ids, attention_mask, labels = batch  
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            predictions = torch.argmax(outputs, dim=1)
            total_acc += (predictions == labels).sum().item()

    print(f'Test loss: {total_loss/len(test_loader)} Test acc: {total_acc/len(test_set)*100}%')

In [ ]:
for epoch in range(3):
    train(model, optimizer, train_loader, criterion)
    evaluate(model, test_loader, criterion)

# Salvando o modelo
torch.save(model.state_dict(), ' sentiment_model.pt')